In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/liar-fake-news-dataset/test.tsv
/kaggle/input/liar-fake-news-dataset/README
/kaggle/input/liar-fake-news-dataset/train.tsv
/kaggle/input/liar-fake-news-dataset/valid.tsv


In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt 


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
BASE_PATH = ''

In [4]:
df = pd.read_csv('/kaggle/input/liar-fake-news-dataset/train.tsv', sep='\t')

In [5]:
df.head()

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [6]:
df.shape

(10239, 14)

In [7]:
for col in df.columns:
    print(col)

2635.json
false
Says the Annies List political group supports third-trimester abortions on demand.
abortion
dwayne-bohac
State representative
Texas
republican
0
1
0.1
0.2
0.3
a mailer


In [8]:
df.columns=['id', 'label', 'statement', 'subjects','speaker', 'speaker_job_title', 'state_info', 'party_affilation',
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement) 
           ]
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [9]:
df.value_counts('label')

label
half-true      2114
false          1994
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
dtype: int64

In [10]:
!pip install -U deep-translator

In [11]:
from deep_translator import GoogleTranslator

In [12]:
translated = GoogleTranslator(source='auto', target='ne').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig
translated

'यसलाई जारी राख्नुहोस्, तपाईं अद्भुत हुनुहुन्छ'

In [13]:
random =  df.sample(20)
random['statement'].iloc[0]

'Here in Virginia, we have the opportunity to become the first state on the East Coast to explore for and produce oil and natural gas offshore.'

In [14]:
translated = GoogleTranslator(source='auto', target='ne').translate(random['statement'].iloc[0])  # output -> Weiter so, du bist großartig
translated

'यहाँ भर्जिनियामा, हामीसँग पूर्वी तटमा तेल र प्राकृतिक ग्यास अपतटीय अन्वेषण र उत्पादन गर्ने पहिलो राज्य बन्ने अवसर छ।'

In [15]:
random['translated_value'] = random['statement'].apply(lambda x: GoogleTranslator(source='auto', target='ne').translate(x))


In [16]:
random['translated_value']

2899     यहाँ भर्जिनियामा, हामीसँग पूर्वी तटमा तेल र प्...
2770     उनले कांग्रेसको लागि आफ्नो उम्मेदवारी घोषणा गर...
3976     टेक्सास किशोर गर्भावस्थामा तेस्रो स्थानमा छ......
9672     राष्ट्रपति बराक ओबामाले आफ्नो आफ्नै (स्वास्थ्य...
5580     मार्क वार्नरले ह्यारी रीडलाई लाइन डाउन समर्थन ...
4911     ओबामा प्रशासनले जासुसी ऐनलाई पत्रकारहरूलाई चुह...
732      IRS सँग स्वास्थ्य सेवालाई अस्वीकार गर्ने वा ढि...
3685     मेरी ल्यान्ड्रीउले तपाईंको बन्दुक अधिकार खोस्न...
6494     कोरी स्टीवर्ट नोभेम्बर 2006 मा अध्यक्ष बनेदेखि...
7685     गर्भावस्थाको आठौंदेखि दशौं हप्तामा सोनोग्राम आ...
6702     हाम्रो औसत निजी क्षेत्रका कर्मचारीहरूले आफ्नो ...
1821     मार्को रुबियो भन्छन् कि अपराधीहरूले उनीहरूको म...
10225    (मेरो कार्यकालको) सबैभन्दा गर्वको उपलब्धि भनेक...
2355     जोन म्याककेनले भनेका छन् कि अर्थव्यवस्था "उनको...
7088     अब धेरैजसो अमेरिकीहरू हाम्रा बच्चाहरूको भविष्य...
1397     स्वास्थ्य हेरचाह कानूनको कारण, 2017 सम्म, हामी...
2029     शर्ली शेरोडले यस बारे फक्समा कसैले एक शब्द बोल.

In [17]:
random['retranslated_value'] = random['translated_value'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))


In [18]:
random['retranslated_value'].iloc[0]

'Here in Virginia, we have the opportunity to be the first state on the East Coast to explore and produce offshore oil and natural gas.'

In [19]:
random['statement'].iloc[2]

'Texas ranks third in teen pregnancies... and it is No. 1 in repeat teen pregnancies.'

In [20]:
index=2
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

Texas ranks third in teen pregnancies... and it is No. 1 in repeat teen pregnancies. 
 टेक्सास किशोर गर्भावस्थामा तेस्रो स्थानमा छ... र यो दोहोरिने किशोर गर्भावस्थामा नम्बर 1 हो। 
 Texas ranks 3rd in teen pregnancies... and it's No. 1 in repeat teen pregnancies.


In [21]:
index=3
print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
random["retranslated_value"].iloc[index])

Says President Barack Obama has amended, delayed, or repealed 19 components of his very own (health care) law. 
 राष्ट्रपति बराक ओबामाले आफ्नो आफ्नै (स्वास्थ्य हेरचाह) कानूनको 19 भागहरू परिमार्जन, ढिलाइ वा खारेज गरेको बताउँछन्। 
 President Barack Obama says he has modified, delayed or repealed 19 pieces of his own (health care) law.


In [22]:
GoogleTranslator(source='auto', target='ne').translate('1919 मा थप किफायती')

'1919 मा थप किफायती'

In [23]:
!pip install inflect


In [24]:
from deep_translator import GoogleTranslator
import inflect

def translate(text, target_lang):
    # Convert numbers to text using the inflect library
    p = inflect.engine()
    words = text.split()
    for i, word in enumerate(words):
        if word.isnumeric():
            words[i] = p.number_to_words(word)
    text = ' '.join(words)
    
    # Translate the text using GoogleTranslator
    translated_text = GoogleTranslator(source='auto', target=target_lang).translate(text)
    
    return translated_text

translate("We, the bishops of the United States—can you believe it—came out in 1919 for more affordable, more comprehensive, more universal health care",'ne')


'हामी, संयुक्त राज्य अमेरिकाका विशपहरू - के तपाइँ यसलाई विश्वास गर्न सक्नुहुन्छ - अधिक किफायती, थप व्यापक, अधिक विश्वव्यापी स्वास्थ्य सेवाको लागि एक हजार, नौ सय उन्नाइसमा बाहिर आयो।'

In [25]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.5 MB/s eta 0:00:00


In [26]:
from deep_translator import GoogleTranslator
from num2words import num2words

def translate_with_numbers(text, target_lang, reverse=False):
    if not reverse:
        # Split the sentence into words
        words = text.split()

        # Convert any numeric words to text using the num2words library
        for i, word in enumerate(words):
            if word.isnumeric():
                words[i] = num2words(int(word), lang='hi')

        # Join the words back into a sentence and translate it using GoogleTranslator
        modified_text = ' '.join(words)
        translated_text = GoogleTranslator(source='auto', target=target_lang).translate(modified_text)

        return translated_text
    else:
        # Translate the text back to the original language
        translated_text = GoogleTranslator(source=target_lang, target='en').translate(text)

        # Replace any numeric text with their corresponding numerical values
        words = translated_text.split()
        for i, word in enumerate(words):
            try:
                numeric_value = int(num2words(word, lang='en'))
                words[i] = str(numeric_value)
            except ValueError:
                pass

        # Join the words back into a sentence and return it
        modified_text = ' '.join(words)
        return modified_text

translate_with_numbers('हामी, संयुक्त राज्य अमेरिकाका विशपहरू - के तपाइँ यसलाई विश्वास गर्न सक्नुहुन्छ - अधिक किफायती, थप व्यापक, अधिक विश्वव्यापी स्वास्थ्य सेवाको लागि एक हजार, नौ सय उन्नाइसमा बाहिर आयो।','en')

'We, the bishops of the United States—can you believe it—came out in nineteen nineteen for more affordable, more comprehensive, more universal health care.'

In [27]:
import string

def removepunctuation(sentence):
  #iterate over every single character to see if it is a punctuation or not, and then concatenate them using "join"
   punctuationfree="".join([i for i in sentence if i not in string.punctuation])
   return punctuationfree

removepunctuation('We, the bishops of the United States—can you believe it—came out in nineteen nineteen for more affordable, more comprehensive, more universal health care.')

'We the bishops of the United States—can you believe it—came out in nineteen nineteen for more affordable more comprehensive more universal health care'

In [28]:
import string

def removepunctuation(sentence):
   new_text = sentence.replace("—", " ") # "—" in the text was not equivalent to "-" so we replcaed "—" with a space
   print 
  #iterate over every single character to see if it is a punctuation or not, and then concatenate them using "join"
   punctuationfree="".join([i for i in new_text if i not in string.punctuation])
   return punctuationfree

removepunctuation('We, the bishops of the United States—can you believe it—came out in nineteen nineteen for more affordable, more comprehensive, more universal health care.')


'We the bishops of the United States can you believe it came out in nineteen nineteen for more affordable more comprehensive more universal health care'